# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [32]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords


## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string

subset = doj[doj['id'] == '17-1204']['contents']

pharma = subset.str.cat(sep=' ')


### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [69]:
## A: your code here to restrict to alpha

def preprocess_text(text):

    punctuation = set(string.punctuation)
    
    words = text.split()
    
    cleaned_words = [''.join(char for char in word if char.isalpha()) for word in words]
    
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text

preprocess_text(pharma)


'The founder and majority owner of Insys Therapeutics Inc was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain More than  Americans died of synthetic opioid overdoses last year and millions are addicted to opioids And yet some medical professionals would rather take advantage of the addicts than try to help them said Attorney General Jeff Sessions This Justice Department will not tolerate this We will hold accountable anyone  from street dealers to corporate executives  who illegally contributes to this nationwide epidemic And under the leadership of President Trump we are fully committed to defeating this threat to the American peopleJohn N Kapoor  of Phoenix Ariz a current member of the Board of Directors of Insys was arrested this morning in Arizona and charged with RICO conspiracy as well as other felonies including con

In [5]:
## B: your code here for part of speech tagging

tokens = word_tokenize(pharma) # Generate list of tokens

tokens_pos = pos_tag(tokens) # generate part of speech tags for those tokens

In [6]:
## C: your code here to extract adjectives 

adj = [word for word, pos in tokens_pos if pos.startswith('JJ')]


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [7]:
## your code here for part A

## resetting pharma 

subset_new = doj[doj['id'] == '17-1204']['contents']

pharma_new = subset.str.cat(sep=' ')

## using spacy 

spacy_pharma = nlp(pharma_new)

for one_tok in spacy_pharma.ents:
    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

Entity: Insys Therapeutics Inc.; NER tag: ORG
Entity: today; NER tag: DATE
Entity: Fentanyl; NER tag: PERSON
Entity: More than 20,000; NER tag: CARDINAL
Entity: Americans; NER tag: NORP
Entity: last year; NER tag: DATE
Entity: millions; NER tag: CARDINAL
Entity: Jeff Sessions; NER tag: PERSON
Entity: This Justice Department; NER tag: ORG
Entity: Trump; NER tag: PERSON
Entity: American; NER tag: NORP
Entity: ”John N. Kapoor; NER tag: PERSON
Entity: 74; NER tag: DATE
Entity: Phoenix; NER tag: GPE
Entity: Ariz.; NER tag: GPE
Entity: the Board of Directors; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: this morning; NER tag: TIME
Entity: Arizona; NER tag: GPE
Entity: RICO; NER tag: LAW
Entity: Kapoor; NER tag: PERSON
Entity: Executive; NER tag: ORG
Entity: Board; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: Phoenix; NER tag: GPE
Entity: today; NER tag: DATE
Entity: U.S.; NER tag: GPE
Entity: District Court; NER tag: ORG
Entity: Boston; NER tag: GPE
Entity: a later date; NER tag: DAT

In [8]:
## your code here for part B

law_ents = set(ent.text for ent in spacy_pharma.ents if ent.label_ == 'LAW')

print(law_ents)


{'the Controlled Substances Act', 'RICO'}


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [ ]:
## C: summarizing RICO

# RICO, or the Racketeer Influenced and Corrupt Organizations, was a law passed in the 1970s primarily intended to target mafia cases, but 
# the flexibility of its statuatory language (engagement in a pattern of criminal activity connected to an enterprise) has allowed it to be 
# applied to cases involving drug cartels, corportations, and even politicians

# Link to Source Website: https://www.justia.com/criminal/docs/rico/

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## D: extracting possible sentence lengths 

date_ents = [ent.text for ent in spacy_pharma.ents if ent.label_ == 'DATE' and re.search(r'\b(?:year|years)\b', ent.text, re.IGNORECASE)]

print(date_ents)

['last year', '20 years', 'three years', 'five years', 'three years']


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [ ]:
## E: your code here



## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [10]:
## your code here for subsetting

doj_subset = doj[doj["topics_clean"].isin(["Civil Rights", "Hate Crimes", "Project Safe Childhood"])]

doj_subset.shape

(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [78]:

def clean_and_analyze_sentiment(text):
    # Remove named entities (assuming they are in the format of word_word)
    cleaned_text = re.sub(r'\b\w+_\w+\b', '', text)
    
    # Initialize the sentiment analyzer
    sent = SentimentIntensityAnalyzer()
    
    # Score the sentiment of the cleaned text
    sentiment_scores = sent.polarity_scores(cleaned_text)
    
    # Return the length-four sentiment dictionary
    return sentiment_scores

# IS IT RIGHT TO ASSUME THAT THE SENTENCES ARE JUST IN THE FORMAT OF WORD_WORD 

In [80]:
## B: your code here 

subset_3 = doj_subset[doj_subset['id'] == '17-1235']['contents']

pharma_3 = subset_3.str.cat(sep=' ')

pharma_3

## Applying this to a specific string 

results = [clean_and_analyze_sentiment(pharma_3) for contents in doj_subset]

results

## OFFICE HOURS QUESTION: DID I TAKE THE RIGHT APPROACH HERE? MAIN ISSUE IS THE PHARMA 3 THING


'A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened.\xa0  \xa0 James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. \xa0 Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. \xa0 “Every citizen has the right to due process and protection from unreasonable force, and

[{'neg': 0.169, 'neu': 0.763, 'pos': 0.068, 'compound': -0.9893},
 {'neg': 0.169, 'neu': 0.763, 'pos': 0.068, 'compound': -0.9893},
 {'neg': 0.169, 'neu': 0.763, 'pos': 0.068, 'compound': -0.9893},
 {'neg': 0.169, 'neu': 0.763, 'pos': 0.068, 'compound': -0.9893},
 {'neg': 0.169, 'neu': 0.763, 'pos': 0.068, 'compound': -0.9893},
 {'neg': 0.169, 'neu': 0.763, 'pos': 0.068, 'compound': -0.9893}]

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [98]:
## C: working with the four sentiment scores 

doj_subset_wscore = doj_subset.copy()

sentiment_scores = doj_subset_wscore['contents'].apply(clean_and_analyze_sentiment)
doj_subset_wscore['neg'] = sentiment_scores.apply(lambda x: x['neg'])
doj_subset_wscore['pos'] = sentiment_scores.apply(lambda x: x['pos'])
doj_subset_wscore['neu'] = sentiment_scores.apply(lambda x: x['neu'])
doj_subset_wscore['compound'] = sentiment_scores.apply(lambda x: x['compound'])

## sorting from highest to lowest

doj_subset_wscore = doj_subset_wscore.sort_values(by='neg', ascending=False)

# print of most negative press releases
top_neg = doj_subset_wscore.head(2)

print(top_neg[['id', 'contents', 'neg']])

           id  \
11593  16-718   
329    14-248   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [97]:
## D: agg and find the mean compound score by topic

print("Mean Compound Sentiment Score by Topic", doj_subset_wscore.groupby("topics_clean")['compound'].agg('mean'))

Mean Compound Sentiment Score by Topic topics_clean
Civil Rights              0.154595
Hate Crimes              -0.884882
Project Safe Childhood   -0.245364
Name: compound, dtype: float64


In [ ]:
## E: interpreting results 

# We see variation in scores because of the connotations of the three categories. There is an overwhelmingly positive connotation associated with
# civil rights, so it makes sense the the sentiment score is positive. There is a strong negative connotation associated with hate crimes, thus
# it makes sense that the sentiment scores for hate crimes is very close to -1. Project Safe Childhood is less obviously positive or negative by 
# looking at the title of the project alone, so a slightly negative sentiment score would make sense. 


## OH QUESTION: WHERE DO I FIND WHERE PROJECT SAFE CHILDHOOD MEANS/WHAT THEY DO? 


# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [154]:
subset_4 = doj_subset_wscore[doj_subset_wscore['id'] == '16-718']['contents']

pharma_4 = subset_3.str.cat(sep=' ')

pharma_4

'A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened.\xa0  \xa0 James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. \xa0 Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. \xa0 “Every citizen has the right to due process and protection from unreasonable force, and

In [121]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]


In [160]:
## A: writing a function

## convert to lowercase and a list
corpus_lower = doj_subset_wscore.contents.str.lower().to_list()

## use wordpunct tokenize and filter out with one
example_listing = corpus_lower[3]
wordpunct_tokenize(example_listing)
nostop_listing = [word 
                for word in wordpunct_tokenize(example_listing) 
                if word not in custom_doj_stopwords]

def process_step1(text):   
    try:
        nostop_listing1 = [word for word in wordpunct_tokenize(example_listing)
                          if word not in custom_doj_stopwords]
        clean_listing = [porter.stem(word) for word in nostop_listing1
                        if word.isalpha() 
                        and len(word) > 4]
        clean_listing_str = " ".join(clean_listing)
        return(clean_listing_str)
    except:
        return("")

results_2 = [process_step1(pharma_4) for contents in doj_subset_wscore['contents']]

results_2

# OFFICE HOURS QUESTION: ISSUE WITH PROCESSED_TEXT COLUMN 


['washington',
 '–',
 'the',
 'justice',
 'department',
 'announced',
 'today',
 'that',
 'sean',
 'popejoy',
 ',',
 '19',
 ',',
 'of',
 'green',
 'forest',
 ',',
 'ark',
 '.,',
 'pleaded',
 'guilty',
 'in',
 'federal',
 'court',
 'to',
 'one',
 'count',
 'of',
 'committing',
 'a',
 'federal',
 'hate',
 'crime',
 'and',
 'one',
 'count',
 'of',
 'conspiring',
 'to',
 'commit',
 'a',
 'federal',
 'hate',
 'crime',
 '.',
 'this',
 'is',
 'the',
 'first',
 'conviction',
 'for',
 'a',
 'violation',
 'of',
 'the',
 'matthew',
 'shepard',
 'and',
 'james',
 'byrd',
 'jr',
 '.',
 'hate',
 'crimes',
 'prevention',
 'act',
 ',',
 'which',
 'was',
 'enacted',
 'in',
 'october',
 '2009',
 '.',
 'information',
 'presented',
 'during',
 'the',
 'plea',
 'hearing',
 'established',
 'that',
 'in',
 'the',
 'early',
 'morning',
 'hours',
 'of',
 'june',
 '20',
 ',',
 '2010',
 ',',
 'popejoy',
 'admitted',
 'that',
 'he',
 'was',
 'part',
 'of',
 'a',
 'conspiracy',
 'to',
 'threaten',
 'and',
 'injure

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [169]:
## B: using apply and creating a new column 

doj_subset_wscore['processed_text'] = doj_subset_wscore['contents'].apply(process_step1)


doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,neg,pos,neu,compound,processed_text
11593,16-718,Three Mississippi Correctional Officers Indicted for Inmate Assault and Cover-Up,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",2016-06-21T00:00:00-04:00,Civil Rights,"Civil Rights Division; Civil Rights - Criminal Section; USAO - Mississippi, Northern",0.259,0.028,0.713,-0.9968,
329,14-248,Albuquerque Man Charged with Federal Hate Crime Related to Anti-Semitic Threats Against Businesswoman,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",2014-03-10T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,0.257,0.044,0.699,-0.9943,
572,13-312,Aryan Brother Inmate Sentenced for Federal Hate Crime for Assaulting Fellow Inmate,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believ

In [167]:
## C: printing id, contents, and processed_text for examples 

row_16718 = doj_subset_wscore[doj_subset_wscore["id"] == "16-718"]
row_16217 = doj_subset_wscore[doj_subset_wscore["id"] == "16-217"]

print("Press Release, 16-718", row_16718[['id', 'contents', 'processed_text']])
print("Press Release, 16-217", row_16217[['id', 'contents', 'processed_text']])



Press Release, 16-718            id  \
11593  16-718   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [183]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [185]:
# A: create_dtm 

dtm_nopre = create_dtm(list_of_strings = doj_subset_wscore.contents,
                metadata = 
                doj_subset_wscore[['id', 'compound', 'topics_clean']])


,index,id,compound,topics_clean,00,000,0082,00pm,01,0225,...,zeeman,zero,zionism,zobel,zone,zones,zoning,zoo,zunggeemoge,zwengel
0,11593,16-718,-0.9968,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,329,14-248,-0.9943,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,572,13-312,-0.9980,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,501,11-626,-0.9985,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10635,11-1531,-0.9968,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,11064,16-471,0.8779,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,11065,16-294,0.9442,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,6787,17-132,0.9905,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,1857,17-271,0.9949,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
## B: print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

top_5 = dtm_nopre['compound'].quantile(0.95)

five_percent = dtm_nopre[dtm_nopre['compound'] >= top_5]

word_counts = five_percent.drop(columns=['id', 'compound', 'topics_clean']).sum()

top_10 = word_counts.sort_values(ascending=False).head(10)

print("Top 10 words for press releases with compount sentiment in the top 5%", top_10)


Top 10 words for press releases with compount sentiment in the top 5% index         226020
the             1769
and              894
of               869
to               683
in               500
that             321
with             305
department       296
for              279
dtype: int64


In [204]:
## C: print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

low_5 = dtm_nopre['compound'].quantile(0.05)

five_below = dtm_nopre[dtm_nopre['compound'] <= low_5]

word_counts_2 = five_below.drop(columns=['id', 'compound', 'topics_clean']).sum()

low_10 = word_counts_2.sort_values(ascending = False).head(10)

print("Top 10 words for press releases with compount sentiment in the top 5%", low_10)

Top 10 words for press releases with compount sentiment in the top 5% index       292925
the           1858
of            1057
and           1025
to             824
in             578
that           319
for            260
attorney       198
was            184
dtype: int64


In [228]:
## D: Print the top 10 words for press releases in each of the three `topics_clean`

grouped = dtm_nopre.groupby('topics_clean')

def top_10_by_group(topics_clean): 
    words = group.drop(columns = ['id', 'compound', 'topics_clean']).sum()
    top_words = words.sort_values(ascending = False).head(10)
    return top_10_by_group

for topic, group in grouped:
    print(f"Top 10 words for press releases in '{topic}':")
    print(top_10_by_group(grouped))
    print()



Top 10 words for press releases in 'Civil Rights':
<function top_10_by_group at 0x28787b9c0>

Top 10 words for press releases in 'Hate Crimes':
<function top_10_by_group at 0x28787b9c0>

Top 10 words for press releases in 'Project Safe Childhood':
<function top_10_by_group at 0x28787b9c0>



## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [ ]:
# your code here

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [ ]:
## your code here to get doc-level topic probabilities 

In [ ]:
## your code here to add those topic probabilities to the dataframe

In [ ]:
## your code here to summarize the topic proportions for each of the topics_clean 

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [ ]:
## your code here 

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [ ]:
# your code here

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 